In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import os

# 設置文件夾保存圖像
output_folder = "trace_images"
os.makedirs(output_folder, exist_ok=True)

In [2]:
# 設定範圍和條件
import glob

# 將日期定義為變量
date = "201010"
dtype = "test_data"
start_date = "2010-10-22"
end_date = "2010-10-31"

# 經緯度設定
extent = [108, 124, 10, 25]

# 讀取數據文件並分類為白天和夜晚
day_files = glob.glob(f"{dtype}/{date}*day*.nc")   # 選擇白天的文件
night_files = glob.glob(f"{dtype}/{date}*night*.nc") # 選擇夜晚的文件


In [3]:
day_files

['test_data/20101017141119-NCEI-L3C_GHRSST-SSTskin-AVHRR_Pathfinder-PFV5.3_NOAA18_G_2010290_day-v02.0-fv01.0.nc',
 'test_data/20101018140028-NCEI-L3C_GHRSST-SSTskin-AVHRR_Pathfinder-PFV5.3_NOAA18_G_2010291_day-v02.0-fv01.0.nc',
 'test_data/20101020230220-NCEI-L3C_GHRSST-SSTskin-AVHRR_Pathfinder-PFV5.3_NOAA18_G_2010293_day-v02.0-fv01.0.nc',
 'test_data/20101031140920-NCEI-L3C_GHRSST-SSTskin-AVHRR_Pathfinder-PFV5.3_NOAA18_G_2010304_day-v02.0-fv01.0.nc',
 'test_data/20101024134308-NCEI-L3C_GHRSST-SSTskin-AVHRR_Pathfinder-PFV5.3_NOAA18_G_2010297_day-v02.0-fv01.0.nc',
 'test_data/20101016142214-NCEI-L3C_GHRSST-SSTskin-AVHRR_Pathfinder-PFV5.3_NOAA18_G_2010289_day-v02.0-fv01.0.nc',
 'test_data/20101029144628-NCEI-L3C_GHRSST-SSTskin-AVHRR_Pathfinder-PFV5.3_NOAA18_G_2010302_day-v02.0-fv01.0.nc',
 'test_data/20101030225600-NCEI-L3C_GHRSST-SSTskin-AVHRR_Pathfinder-PFV5.3_NOAA18_G_2010303_day-v02.0-fv01.0.nc',
 'test_data/20101027140458-NCEI-L3C_GHRSST-SSTskin-AVHRR_Pathfinder-PFV5.3_NOAA18_G_2010

In [ ]:
# 初始化經緯度數據
all_longitudes = []
all_latitudes = []

# 遍歷每個文件
for file in day_files:
    if not os.path.exists(file):  # 確保文件存在
        print(f"文件 {file} 不存在，跳過此文件")
        continue
    
    try:
        # 打開 NetCDF 文件
        data = xr.open_dataset(file)

        # 確認經緯度變量名稱
        if 'lon' not in data or 'lat' not in data:
            print(f"文件 {file} 缺少經緯度變量，跳過")
            continue

        longitude = data['lon'].values
        latitude = data['lat'].values

        # 如果經緯度是一維數組，生成匹配網格
        if len(longitude.shape) == 1 and len(latitude.shape) == 1:
            longitude, latitude = np.meshgrid(longitude, latitude)

        # 展平數據
        longitude = longitude.flatten()
        latitude = latitude.flatten()

        # 過濾 NaN 值
        mask = ~np.isnan(longitude) & ~np.isnan(latitude)
        longitude = longitude[mask]
        latitude = latitude[mask]

        # 添加到總列表
        all_longitudes.append(longitude)
        all_latitudes.append(latitude)

    except Exception as e:
        print(f"處理文件 {file} 時發生錯誤: {e}")
        continue

# 合併所有數據
if len(all_longitudes) == 0 or len(all_latitudes) == 0:
    print("無有效數據，無法繪製軌跡")
else:
    all_longitudes = np.concatenate(all_longitudes)
    all_latitudes = np.concatenate(all_latitudes)

    # 繪製地圖
    fig = plt.figure(figsize=(12, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # 添加地圖特徵
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.set_global()  # 顯示全球地圖

    # 繪製軌跡
    ax.scatter(all_longitudes, all_latitudes, s=1, color='blue', transform=ccrs.PlateCarree(), label='Satellite Tracks')

    # 添加標題與圖例
    plt.title('Satellite Tracks Over Multiple Days')
    plt.legend()
    plt.show()